In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

df_train = pd.read_csv("E:\\leonhardt\\mayi\\data\\train.csv")

In [2]:
df_train=df_train.loc[df_train['label']!=-1]
df_train = df_train.fillna(0)
train_set = df_train[df_train.loc[:, "date"]<=20171005]
eva_set = df_train[df_train.loc[:, "date"]>20171005]
train_label = train_set.loc[:, "label"]
eva_label = eva_set.loc[:, "label"]
train_set.drop(["id","date", "label"], axis=1, inplace=True)
eva_set.drop(["id","date", "label"], axis=1, inplace=True)

D:\anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [29]:
from sklearn.metrics import roc_auc_score,roc_curve
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1], True

In [25]:
import lightgbm as lgb
lgb_train=lgb.Dataset(train_set,train_label,free_raw_data=False)
lgb_valid=lgb.Dataset(eva_set,eva_label,reference=lgb_train,free_raw_data=False)
params = {'boosting_type': 'gbdt',
          'objective': 'binary', 
          'metric': 'auc', 
          'device': 'cpu', 
          'verbose': 0, 
          'learning_rate': 0.01, 
          'bagging_fraction': 0.8,
          'bagging_freq': 5,
          'feature_fraction': 0.9,
          'lambda_l1': 0.006, 
          'num_leaves': 70
#           'early_stopping_round': 30
}

In [26]:
gbm=lgb.train(params, lgb_train, num_boost_round=3000, valid_sets=lgb_valid, valid_names='valid_set', feval=atec_metric, verbose_eval=30)

[30]	valid_set's auc: 0.952907	valid_set's atec_metric: 0.456629
[60]	valid_set's auc: 0.959381	valid_set's atec_metric: 0.463488
[90]	valid_set's auc: 0.964059	valid_set's atec_metric: 0.467638
[120]	valid_set's auc: 0.967764	valid_set's atec_metric: 0.478109
[150]	valid_set's auc: 0.969953	valid_set's atec_metric: 0.485855
[180]	valid_set's auc: 0.971357	valid_set's atec_metric: 0.487819
[210]	valid_set's auc: 0.972258	valid_set's atec_metric: 0.48972
[240]	valid_set's auc: 0.973583	valid_set's atec_metric: 0.492381
[270]	valid_set's auc: 0.974806	valid_set's atec_metric: 0.495137
[300]	valid_set's auc: 0.975532	valid_set's atec_metric: 0.495882
[330]	valid_set's auc: 0.97605	valid_set's atec_metric: 0.498115
[360]	valid_set's auc: 0.976426	valid_set's atec_metric: 0.499034
[390]	valid_set's auc: 0.976991	valid_set's atec_metric: 0.502043
[420]	valid_set's auc: 0.97774	valid_set's atec_metric: 0.501964
[450]	valid_set's auc: 0.97837	valid_set's atec_metric: 0.50282
[480]	valid_set's 

KeyboardInterrupt: 

In [6]:
df_test = pd.read_csv("E:\\leonhardt\\mayi\\data\\test_a.csv")
df_test = df_test.fillna(0)
df_test.drop(["date","id"],axis=1, inplace=True)

In [7]:
y_pred = gbm.predict(df_test)

In [9]:
y_pred.min()

0.0014065366055244087

In [11]:
y_pred.max()

0.900688296638455

In [12]:
np.savetxt("E:\\leonhardt\\mayi\\mayi\\ipython\\prediction/lgb_half_fill0_600.txt", y_pred, delimiter=",")

In [13]:
import xgboost as xgb

In [14]:
xgb_train = xgb.DMatrix(train_set, label=train_label)
xgb_vaild = xgb.DMatrix(eva_set, label=eva_label)

In [31]:
def atec_metric(preds, train_data):
    labels = train_data.get_label()
    fpr,tpr,threshhold=roc_curve(labels, preds)
    return 'atec_metric', 0.4*tpr[fpr<=0.001][-1] + 0.3*tpr[fpr<=0.005][-1] + 0.3*tpr[fpr<=0.01][-1]

In [32]:
params={
'booster':'gbtree',
'objective': 'binary:logistic', #单分类的问题
'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':12, # 构建树的深度，越大越容易过拟合
'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.007, # 如同学习率
'seed':10,
'nthread':7,# cpu 线程数
'eval_metric': 'auc',
'n_jobs': -1
}
plst = list(params.items())
num_rounds = 1000 # 迭代次数
watchlist = [(xgb_train, 'train'),(xgb_vaild, 'val')]

#训练模型并保存
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train, num_rounds, watchlist, feval = atec_metric,verbose_eval=30)

[0]	train-auc:0.958632	val-auc:0.955429	train-atec_metric:0.534378	val-atec_metric:0.365262
[30]	train-auc:0.982733	val-auc:0.968775	train-atec_metric:0.678654	val-atec_metric:0.486266
[60]	train-auc:0.983481	val-auc:0.966919	train-atec_metric:0.687726	val-atec_metric:0.484033
[90]	train-auc:0.98463	val-auc:0.968104	train-atec_metric:0.69308	val-atec_metric:0.489165
[120]	train-auc:0.985518	val-auc:0.968407	train-atec_metric:0.704011	val-atec_metric:0.491399
[150]	train-auc:0.986338	val-auc:0.96914	train-atec_metric:0.714237	val-atec_metric:0.494297
[180]	train-auc:0.987087	val-auc:0.970784	train-atec_metric:0.721725	val-atec_metric:0.495185
[210]	train-auc:0.987571	val-auc:0.97161	train-atec_metric:0.730745	val-atec_metric:0.498796
[240]	train-auc:0.988087	val-auc:0.972684	train-atec_metric:0.737545	val-atec_metric:0.500143
[270]	train-auc:0.988593	val-auc:0.973388	train-atec_metric:0.745481	val-atec_metric:0.500935
[300]	train-auc:0.989199	val-auc:0.974243	train-atec_metric:0.753538	

In [34]:
xgb_test = xgb.DMatrix(df_test)
y_preds_xgb = model.predict(xgb_test)

In [37]:
y_preds_xgb.min()

0.0005163161

In [39]:
y_preds_xgb.max()

0.91339743

In [40]:
np.savetxt("E:\\leonhardt\\mayi\\mayi\\ipython\\prediction/xgb_fil0_1000.txt", y_preds_xgb, delimiter=",")

In [41]:
params={
'booster':'gbtree',
'objective': 'binary:logistic', #单分类的问题
'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
'max_depth':12, # 构建树的深度，越大越容易过拟合
'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
'subsample':0.7, # 随机采样训练样本
'colsample_bytree':0.7, # 生成树时进行的列采样
'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
'eta': 0.007, # 如同学习率
'seed':10,
'nthread':7,# cpu 线程数
'eval_metric': 'auc',
'n_jobs': -1
}
plst = list(params.items())
num_rounds = 330 # 迭代次数
watchlist = [(xgb_train, 'train'),(xgb_vaild, 'val')]

#训练模型并保存
# early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train, num_rounds, watchlist, feval = atec_metric,verbose_eval=30)

[0]	train-auc:0.958632	val-auc:0.955429	train-atec_metric:0.534378	val-atec_metric:0.365262
[30]	train-auc:0.982733	val-auc:0.968775	train-atec_metric:0.678654	val-atec_metric:0.486266
[60]	train-auc:0.983481	val-auc:0.966919	train-atec_metric:0.687726	val-atec_metric:0.484033
[90]	train-auc:0.98463	val-auc:0.968104	train-atec_metric:0.69308	val-atec_metric:0.489165
[120]	train-auc:0.985518	val-auc:0.968407	train-atec_metric:0.704011	val-atec_metric:0.491399
[150]	train-auc:0.986338	val-auc:0.96914	train-atec_metric:0.714237	val-atec_metric:0.494297
[180]	train-auc:0.987087	val-auc:0.970784	train-atec_metric:0.721725	val-atec_metric:0.495185
[210]	train-auc:0.987571	val-auc:0.97161	train-atec_metric:0.730745	val-atec_metric:0.498796
[240]	train-auc:0.988087	val-auc:0.972684	train-atec_metric:0.737545	val-atec_metric:0.500143
[270]	train-auc:0.988593	val-auc:0.973388	train-atec_metric:0.745481	val-atec_metric:0.500935
[300]	train-auc:0.989199	val-auc:0.974243	train-atec_metric:0.753538	

In [42]:
y_preds_xgb330 = model.predict(xgb_test)

In [45]:
np.savetxt("E:\\leonhardt\\mayi\\mayi\\ipython\\prediction/xgb_fil0_330.txt", y_preds_xgb330, delimiter=",")

In [47]:
y_preds_xgb330.min()

0.049839422

In [49]:
y_preds_xgb330.max()

0.8352195